1-[read from text](#from_text)

2-[hand input](#hand_input)

Some designing notes:
- id. **Answers can have the same ids (plotted in the same place) while questions do not**. Some questions will be repeated in different contexts. Yet each answer is unique and the same answer should be regarded a same thing. Unless it's like really simple questions ("yes/no").

In [29]:
import numpy as np
import pandas as pd
import re
import pickle
import os

# from_text

In [26]:
with open('What_is_curiosity.txt','r') as f:
    content = f.readlines()
content = [x.strip('\n') for x in content] # remove whitespace at the end. keep the tab \t to mark level of question

rootlv=0
root = content[0]
nodedist = {'id':['qd%d'%np.random.randint(100000)],'label':[root],'feature':['Central question'],'content':[""],'reference':[""]}


rootlist = [[] for k in range(10)]
rootlist[rootlv] = nodedist['id'][0]
fullnet = []


for line in content[1:]:
    if len(line)<1: # skip empty lines
        continue
    edge = line[line.find('_')+1:line.rfind('_')]
    node = line[line.rfind('_')+2:]
    if edge == 'answer':
        label = node[:node.find('.')]
        if node.find('[')>=0:
            content = node[node.find('.')+1:node.find('[')]
            reference = node[node.find('[')+1:node.find(']')]
        else:
            content = []
            reference = []
        if label not in nodedist['label']: # if an answer has appeared earlier (usually an entity?), should be represented as the same node.            
            nid = 'ad%d'%np.random.randint(100000)
            nodedist['feature'].append('Answer')    
            nodedist['label'].append(label)
            nodedist['id'].append(nid)
            nodedist['content'].append(content)
            nodedist['reference'].append(reference)
            
        else:
            nid = nodedist['id'][nodedist['label'].index(label)]
    else:
        label = node[:(node.find('?')+1)]
        if node.find('[')>=0:
            content = node[node.find('?')+1:node.find('[')]
            reference = node[node.find('[')+1:node.find(']')]
        else:
            content = []
            reference = []
        nid = 'qd%d'%np.random.randint(100000)
        nodedist['feature'].append('Question')
        nodedist['label'].append(label)
        nodedist['id'].append(nid)
        nodedist['content'].append(content)
        nodedist['reference'].append(reference)
    
    level = int((len(line) - len(line.strip('\t')))/len('\t'))
    edgedict = {'start':rootlist[level-1],'end':nid,'label':edge,'id':'l%d'%np.random.randint(100000)}
    fullnet.append(edgedict)    
    rootlist[level]=nid
nodedf = pd.DataFrame(nodedist)
edgedf = pd.DataFrame(fullnet)  

In [30]:
filedir = 'database/curiosity'
os.makedirs(filedir)

In [35]:
pickle.dump([nodedf,edgedf],open(filedir+'/curiosity.p','bw'))

# To-Networkx

In [27]:
# generate text files for nodes
nodestr = '['
for knd in range(len(nodedf)):
    nd=nodedf.iloc[knd]
    nodestr = nodestr + '{"id":"%s","label":"%s","feature":"%s","content":"%s","reference":"%s",}'%(nd['id'],nd['label'],nd['feature'],nd['content'],nd['reference'])
nodestr = nodestr[:-1] #delete last comma
with open(filedir+'/node_curiosity.txt','w') as f:
    f.write(nodestr)

# generate text files for edges
edgestr = '['
for knd in range(len(edgedf)):
    ed=edgedf.iloc[knd]
    edgestr = edgestr + '{"id":"%s","label":"%s","start":"%s","end":"%s",}'%(ed['id'],ed['label'],ed['start'],ed['end'])
edgestr = edgestr[:-1] #delete last comma
with open(filedir+'/edge_curiosity.txt','w') as f:
    f.write(edgestr)

# To-Json 

In [102]:
# styling for the package
rootcolor = 'orange'
anscolor = 'white'

jsonstr='\n"nodes":[\n'
for knode in range(len(nodedist['id'])):
    if nodedist['type'][knode]=='Q':
        jsonstr = jsonstr+'{"id":"%s","loaded":true,"style":{"label":"%s","fillColor":"%s"}},'%(nodedist['id'][knode],nodedist['label'][knode],rootcolor)
    elif nodedist['type'][knode]=='A':
        jsonstr = jsonstr+'{"id":"%s","loaded":true,"style":{"label":"%s","fillColor":"%s"}},'%(nodedist['id'][knode],nodedist['label'][knode],anscolor)
    else:
        set_trace()
        jsonstr = jsonstr+'{"id":"%s","loaded":true,"style":{"label":"%s"}},'%(node['id'],node['label'])
jsonstr = jsonstr+'],\n"links":[\n'

for link in fullnet:
    jsonstr = jsonstr+'{"id":"%s","from":"%s","to":"%s","type":"%s"},'%(link['eid'],link['from'],link['to'],link['edge'])
jsonstr = jsonstr[:-1]    

# todo: more details within each node to be differentiated. only for answers
# also, tiny problem: " in label, can't be normally read. should go to \" for some json thing.

In [103]:
with open('curiosity.json','w') as f:
    f.write(jsonstr)

In [94]:
nodedist.keys()

dict_keys(['type', 'id', 'label'])

In [95]:
nodedist=pd.DataFrame(nodedist)
nodedist

,id,label,type
0,qd5030,What is curiosity?,Q
1,qd63348,What is the time scale of curiosity?,Q
2,ad5412,lifelong (personality trait),A
3,qd37294,Is curiosity a stable trait?,Q
4,ad28604,Probably not,A
5,ad81555,seconds (state),A
6,qd59535,What is the operationalized definition?,Q
7,ad32479,Melbourne Curiosity Inventory (MCI),A
8,ad50922,State-Trait Personality Inventory (STPI),A
9,ad3939,a motivator of exploratory behavior,A


# hand_input 
hand input each node-link pair

In [55]:
# input field
# question structures
defdims = [{'from':'What is the time scale of curiosity?',
               'to':['seconds/within trial','minutes/within trial', 'lifelong/personality trait']},#link list. todo: make it a type of object and add citations
           {'from':'What is the structure of knowledges being seeken for?',
               'to':['few connections/busybody','closely connected/hunter']},
           {'from':'What is the internal state computation that increases motivation?',
               'to':['learning progress','surprise','positive prediction error']}]
defobjs = [{'operdef':'self-reported curiosity level','paper':[],'dimlinks':['seconds/within trial','few connections/busybody']},
           {'operdef':'willingness to stick to the current game','dimlinks':['minutes/within trial','few connections/busybody'],'paper':['Bonawitz et al']},
           {'operdef':'KL divergence of belief state','dimlinks':['surprise'],'paper':[]}]


In [60]:
# convert to json for zoomcharts
allnodes = []
# add root note: the general concept we are talking about
rid='qd%d'%np.random.randint(100000)
allnodes.append({'label':'What is curiosity?','id':rid})
alllinks = []
for dim in defdims:
    # create nodes
    fid = 'qd%d'%np.random.randint(100000)
    fnode = {'label':dim['from'],'id':fid} #todo: add number repetition check
    allnodes.append(fnode)
    # add link from the rootnode
    alllinks.append({'label':'specification','id':'ld%d'%np.random.randint(100000),'from':rid,'to':fid})
    for tond in dim['to']:
        tid = 'a%d'%np.random.randint(100000)
        tonode = {'label':tond,'id':tid}
        allnodes.append(tonode)
        # add links
        alllinks.append({'label':'answer','id':'ld%d'%np.random.randint(100000),'from':fid,'to':tid})
nodedf = pd.DataFrame(allnodes)

#each operational definition object
for defob in defobjs:
    defid = 'ado%d'%np.random.randint(100000)
    allnodes.append({'label':defob['operdef'],'id':defid,'paper':defob['paper']})
    for kl in defob['dimlinks']:
        alllinks.append({'label':'operational_def','id':'lod%d'%np.random.randint(100000),
                        'from':nodedf['id'].iloc[nodedf.index[nodedf['label']==kl][0]],
                        'to':defid})

# styling
rootcolor = 'orange'
anscolor = 'white'

In [72]:
# translate to json
print("\nnode:\n")
for node in allnodes:
    if node['id']==rid:
        print('{"id":"%s","loaded":true,"style":{"label":"%s","fillColor":"%s"}},'%(node['id'],node['label'],rootcolor))
    elif node['id'][0]=='a':
        print('{"id":"%s","loaded":true,"style":{"label":"%s","fillColor":"%s"}},'%(node['id'],node['label'],anscolor))
    else:
        print('{"id":"%s","loaded":true,"style":{"label":"%s"}},'%(node['id'],node['label']))
print("\nlinks:\n")
for link in alllinks:
    print('{"id":"%s","from":"%s","to":"%s","type":"%s"},'%(link['id'],link['from'],link['to'],link['label']))    


node:

{"id":"qd39020","loaded":true,"style":{"label":"What is curiosity?","fillColor":"orange"}},
{"id":"qd68905","loaded":true,"style":{"label":"What is the time scale of curiosity?"}},
{"id":"a60280","loaded":true,"style":{"label":"seconds/within trial","fillColor":"white"}},
{"id":"a79023","loaded":true,"style":{"label":"minutes/within trial","fillColor":"white"}},
{"id":"a90009","loaded":true,"style":{"label":"lifelong/personality trait","fillColor":"white"}},
{"id":"qd75943","loaded":true,"style":{"label":"What is the structure of knowledges being seeken for?"}},
{"id":"a67943","loaded":true,"style":{"label":"few connections/busybody","fillColor":"white"}},
{"id":"a4582","loaded":true,"style":{"label":"closely connected/hunter","fillColor":"white"}},
{"id":"qd42158","loaded":true,"style":{"label":"What is the internal state computation that increases motivation?"}},
{"id":"a50961","loaded":true,"style":{"label":"learning progress","fillColor":"white"}},
{"id":"a45237","loaded":t

In [98]:
# translate to table
fulltable = []
opdf = pd.DataFrame(defobjs,index=[ob['operdef'] for ob in defobjs]) # temp: check dataframe
for dim in defdims:
    table = {}
    for prop in dim['to']:
        x = np.zeros(len(opdf),dtype=int)
        for k in range(len(opdf)):
            if prop in opdf.iloc[k]['dimlinks']:
                x[k]=1
        table[prop]=x
    fulltable.append({'dim':dim['from'],'table':pd.DataFrame(table,index=opdf.index)})

In [99]:
fulltable[0]['table']

,lifelong/personality trait,minutes/within trial,seconds/within trial
self-reported curiosity level,0,0,1
willingness to stick to the current game,0,1,0
KL divergence of belief state,0,0,0
